In [115]:
import dataclasses
from collections import Counter
from typing import Tuple, List, Dict, Union, Set, Iterator, Optional
from enum import Enum, auto
import time

from protein import trypsin
from measurement import read_mgf, PeptideMeasurement
from pyteomics import mass
from common import LYS, BSA


@dataclasses.dataclass
class AminoAcid:
    name: str
    # TODO: [i].mass should include C modifications
    mass: float


@dataclasses.dataclass
class Mod:
    description: str
    mass: float

    def __hash__(self):
        return (self.description, self.mass).__hash__()


@dataclasses.dataclass
class Modification:
    name: str
    mass: float
    count: int


class Peptide:
    beginning: int
    end: int
    seq: str

    _modifications: Dict[str, Tuple[Mod, int]]
    _amino_acids: List[AminoAcid]
    _aas = None
    _mass = None
    _minmass = None
    _maxmass = None

    def __init__(
        self,
        beginning: int,
        end: int,
        seq: str,
        modifications: Dict[str, Tuple[Mod, int]],
    ):
        self.beginning = beginning
        self.end = end
        self.seq = seq
        h2o = mass.calculate_mass(formula="H2O")
        self._amino_acids = [
            AminoAcid(aa, mass.calculate_mass(sequence=aa) - h2o) for aa in seq
        ]
        self._modifications = modifications

    def __getitem__(self, index: int):
        if self.beginning <= index < self.end:
            return self._amino_acids[index - self.beginning]
        return None

    def __iter__(self):
        return range(self.beginning, self.end).__iter__()

    def __add__(self, other):
        if other.beginning != self.end:
            raise ValueError(
                f"Peptides can only be added when they are contiguous. Got {(self.beginning, self.end)} + {other.beginning, other.end} instead."
            )

        merged_mods = self._modifications
        for target, (mod, c2) in other._modifications.items():
            mod2, c1 = merged_mods.setdefault(target, (mod, 0))
            if mod != mod2:
                raise ValueError(
                    f"Peptides can only be added when they have compatible modifications. These two differ at {target}"
                )
            merged_mods[target] = mod, c1 + c2

        return Peptide(self.beginning, other.end, self.seq + other.seq, merged_mods)

    def count(self, amino_acid):
        if self._aas is None:
            self._aas = Counter(self.seq)
        return self._aas[amino_acid]

    @property
    def zwitterion_mass(self):
        if self._mass is None:
            self._mass = mass.calculate_mass(
                sequence=self.seq, ion_type="M", charge=0
            ) - mass.calculate_mass(formula="H2O")
        return self._mass

    @property
    def min_mass(self):
        if self._minmass is None:
            neg = sum(
                m.mass * count for m, count in self.modifications_anywhere if m.mass < 0
            )
            self._minmass = self.zwitterion_mass + neg
        return self._minmass

    @property
    def max_mass(self):
        if self._maxmass is None:
            pos = sum(
                m.mass * count for m, count in self.modifications_anywhere if m.mass > 0
            )
            self._maxmass = self.zwitterion_mass + pos
        return self._maxmass

    @property
    def modifications_anywhere(self) -> Iterator[Tuple[Mod, int]]:
        return (x for x in self._modifications.values())

    def __repr__(self):
        return f"Peptide(beginning={self.beginning}, end={self.end}, seq={self.seq}, modifications={self._modifications})"


In [100]:
measurements = {m.scan: m for m in read_mgf("../data/mgf/190318_LYS_AT_50x_05.mgf")}
list(measurements.items())[:10]

[(3, <measurement.PeptideMeasurement at 0x121a5ee50>),
 (7, <measurement.PeptideMeasurement at 0x121a5eaf0>),
 (9, <measurement.PeptideMeasurement at 0x1245d89a0>),
 (12, <measurement.PeptideMeasurement at 0x12457fd00>),
 (13, <measurement.PeptideMeasurement at 0x12457fcd0>),
 (29, <measurement.PeptideMeasurement at 0x121d95190>),
 (37, <measurement.PeptideMeasurement at 0x121d95bb0>),
 (42, <measurement.PeptideMeasurement at 0x121d95f10>),
 (54, <measurement.PeptideMeasurement at 0x121d95ca0>),
 (57, <measurement.PeptideMeasurement at 0x121f7d9d0>)]

In [116]:
peptides = []
for b, e in trypsin(LYS):
    seq = LYS[b:e]
    met_ox = (Mod("met_ox", 15.9949), sum(aa == "M" for aa in seq))
    mods = {"M": met_ox} if "M" in seq else {}
    peptides.append(Peptide(b, e, seq, modifications=mods))

peptides

[Peptide(beginning=0, end=1, seq=K, modifications={}),
 Peptide(beginning=1, end=5, seq=VFGR, modifications={}),
 Peptide(beginning=5, end=13, seq=CELAAAMK, modifications={'M': (Mod(description='met_ox', mass=15.9949), 1)}),
 Peptide(beginning=13, end=14, seq=R, modifications={}),
 Peptide(beginning=14, end=21, seq=HGLDNYR, modifications={}),
 Peptide(beginning=21, end=33, seq=GYSLGNWVCAAK, modifications={}),
 Peptide(beginning=33, end=45, seq=FESNFNTQATNR, modifications={}),
 Peptide(beginning=45, end=61, seq=NTDGSTDYGILQINSR, modifications={}),
 Peptide(beginning=61, end=68, seq=WWCNDGR, modifications={}),
 Peptide(beginning=68, end=73, seq=TPGSR, modifications={}),
 Peptide(beginning=73, end=96, seq=NLCNIPCSALLSSDITASVNCAK, modifications={}),
 Peptide(beginning=96, end=97, seq=K, modifications={}),
 Peptide(beginning=97, end=112, seq=IVSDGNGMNAWVAWR, modifications={'M': (Mod(description='met_ox', mass=15.9949), 1)}),
 Peptide(beginning=112, end=114, seq=NR, modifications={}),
 Pepti

In [102]:

def within_bounds(reference_mass, measured_mass, ppm_error=10):
    return abs(reference_mass - measured_mass) <= err_margin(reference_mass, ppm_error)


def err_margin(reference_mass, ppm_error=10):
    return (ppm_error / 1e6) * reference_mass


def compute_error(reference_mass, measured_mass):
    return 1e6 * abs(measured_mass - reference_mass) / reference_mass


class State(Enum):
    BEFORE = auto()
    DURING = auto()


def set_tuple(t, i, x):
    return t[:i] + (x,) + t[i + 1 :]


# Pass None when you want to allow to skip a mod
def combine_modifications_2(
    modifications: List[List[Union[Mod, None]]],
    starting_mass: float,
    target_mass: float,
    ppm_error: float = 10,
) -> List[List[Mod]]:
    result = []

    def go(i, current, selection):
        if i == len(modifications):
            if within_bounds(current, target_mass, ppm_error):
                result.append(selection)
        else:
            for m in modifications[i]:
                if m is None:
                    go(i + 1, current, selection)
                else:
                    go(i + 1, current + m.mass, selection + (m,))

    go(0, current=starting_mass, selection=())
    return list(set(result))


# peptide_masses should be digestion peptides with H2O loss
# TODO: Přepsat na čitelnější verzi
def precursor_mass_matches(
    peptides: List[Peptide],
    measurement: PeptideMeasurement,
    alkylation_mass: float,
    max_inter_bonds: int,
    ppm_error: int = 10,
) -> List[str]:
    target = measurement.peptide_mass_estimate
    h2o = mass.calculate_mass(formula="H2O")
    h2 = mass.calculate_mass(formula="H2")

    result = []

    def go(
        i: int,
        current: float,
        min_raw_mass: float,
        max_raw_mass: float,
        selection: Tuple[int, ...],
        state: State,
        inter_bonds_left: int,
        cysteines_before: int,
        cysteines_now: int,
    ) -> None:
        internal_cysteines = cysteines_before + cysteines_now

        # Non-bonded cysteines are alkylated, or modified in another way
        if cysteines_now >= 0:
            max_posibble_mass = max_raw_mass + alkylation_mass * internal_cysteines
            upper_bound = max_posibble_mass + err_margin(max_posibble_mass, ppm_error)

            if target <= upper_bound:
                has_alkylated_cys = internal_cysteines % 2 == 1
                min_possible_mass = min_raw_mass + alkylation_mass * has_alkylated_cys
                lower_bound = min_possible_mass - err_margin(
                    min_possible_mass, ppm_error
                )

                if lower_bound <= target:
                    ranges = list(zip(selection[::2], (selection + (i,))[1::2]))

                    possible_mods: List[List[Mod]] = []

                    for b, e in ranges:
                        for p in peptides[b:e]:
                            for m, count in p.modifications_anywhere:
                                possible_mods += [
                                    [Mod(m.description, m.mass), None]
                                ] * count

                    max_intra_bonds = internal_cysteines // 2
                    for _ in range(max_intra_bonds):
                        possible_mods.append(
                            [Mod("cys_pair_alk", alkylation_mass * 2), None]
                        )

                    seq = "+".join(
                        "".join(p.seq for p in peptides[b:e]) for b, e in ranges
                    )

                    if has_alkylated_cys:
                        # One Cys has to be alkylated, as it can't be in a bond
                        possible_mods.append([Mod("cys_alk", alkylation_mass)])

                    combinations = combine_modifications_2(
                        possible_mods,
                        starting_mass=current,
                        target_mass=target,
                        ppm_error=ppm_error,
                    )

                    for modifications in combinations:
                        total_mass = current + sum(m.mass for m in modifications)

                        alkylated_pairs = sum(
                            m.description == "cys_pair_alk" for m in modifications
                        )
                        intra_bonds = max_intra_bonds - alkylated_pairs
                        inter_bonds = max_inter_bonds - inter_bonds_left

                        result.append(
                            {
                                "sequence": seq,
                                "ranges": ranges,
                                "cysteine_bonds": intra_bonds + inter_bonds,
                                "inter_bonds": inter_bonds,
                                "intra_bonds": intra_bonds,
                                "mass": total_mass,
                                "error": compute_error(total_mass, target),
                                "mods": modifications,
                            }
                        )

        if (
            i == len(peptides)
            or min_raw_mass - err_margin(min_raw_mass, ppm_error) > target
        ):
            # Either we're out of peptides to add
            # Or we're too high and we'll never correct it
            return
        else:
            if state == State.BEFORE:
                # Don't start yet
                go(
                    i + 1,
                    current,
                    min_raw_mass,
                    max_raw_mass,
                    selection,
                    state.BEFORE,
                    inter_bonds_left,
                    cysteines_before,
                    cysteines_now,
                )
            elif (
                state == State.DURING and min(inter_bonds_left, internal_cysteines) > 0
            ):
                # End this run, begin next one
                go(
                    i,
                    current + h2o - h2,
                    min_raw_mass + h2o - h2,
                    max_raw_mass + h2o - h2,
                    selection + (i,),
                    state.BEFORE,
                    inter_bonds_left - 1,
                    internal_cysteines - 1,
                    -1,
                )

            # Take this one, and either begin or continue this run
            go(
                i + 1,
                current + peptides[i].zwitterion_mass,
                min_raw_mass + peptides[i].min_mass,
                max_raw_mass + peptides[i].max_mass,
                selection + (i,) if state == State.BEFORE else selection,
                State.DURING,
                inter_bonds_left,
                cysteines_before,
                cysteines_now + peptides[i].count("C"),
            )

    go(
        0,
        current=h2o,
        min_raw_mass=h2o,
        max_raw_mass=h2o,
        selection=(),
        state=State.BEFORE,
        inter_bonds_left=max_inter_bonds,
        cysteines_before=0,
        cysteines_now=0,
    )

    return result


In [31]:
import csv

FILE_PATH = "../out/precursor_matches_lys_at_2_inter_bonds.csv"

start_time = time.time()

with open(FILE_PATH, "w") as f:
    field_names = [
        "scan",
        "sequence",
        "mass",
        "error",
        "cysteine_bonds",
        "inter_bonds",
        "intra_bonds",
        "mods",
    ]
    writer = csv.DictWriter(f, fieldnames=field_names)
    writer.writeheader()

    for scan, measurement in measurements.items():
        for match in precursor_mass_matches(
            peptides,
            measurement,
            alkylation_mass=57.0214,
            max_inter_bonds=2,
            ppm_error=15,
        ):
            writer.writerow({"scan": scan} | match)
end_time = time.time()

print(f"This takes {end_time - start_time} seconds")

This takes 48.51588582992554 seconds


In [96]:
TAKE = 100

start_time = time.time()

for scan, measurement in list(measurements.items())[:TAKE]:
    for match in sorted(
        precursor_mass_matches(
            peptides,
            measurement,
            alkylation_mass=57.0214,
            max_inter_bonds=2,
            ppm_error=15,
        ),
        key=lambda m: m["sequence"],
    ):
        print(f"{scan}: {match}")

end_time = time.time()
print(f"This takes {end_time - start_time} seconds")

845: {'sequence': 'CELAAAMK+GCR', 'segments': [Peptide(beginning=5, end=13, seq=CELAAAMK, modifications={'M': (Mod(description='met_ox', mass=15.9949), 1)}), Peptide(beginning=125, end=128, seq=GCR, modifications={})], 'cysteine_bonds': 1, 'inter_bonds': 1, 'intra_bonds': 0, 'mass': 1183.51476996307, 'error': 0.17497809442811452, 'mods': (Mod(description='met_ox', mass=15.9949),)}
846: {'sequence': 'RHGLDNYR', 'segments': [Peptide(beginning=13, end=21, seq=RHGLDNYR, modifications={})], 'cysteine_bonds': 0, 'inter_bonds': 0, 'intra_bonds': 0, 'mass': 1029.51042528457, 'error': 0.046330920919352085, 'mods': ()}
848: {'sequence': 'CELAAAMK+GCR', 'segments': [Peptide(beginning=5, end=13, seq=CELAAAMK, modifications={'M': (Mod(description='met_ox', mass=15.9949), 1)}), Peptide(beginning=125, end=128, seq=GCR, modifications={})], 'cysteine_bonds': 1, 'inter_bonds': 1, 'intra_bonds': 0, 'mass': 1183.51476996307, 'error': 0.7012779318072832, 'mods': (Mod(description='met_ox', mass=15.9949),)}


KeyboardInterrupt: 

In [103]:
# 7012, 7013

for sc in [10973]:
    print(sc)

    matches = precursor_mass_matches(
        peptides,
        measurements[sc],
        alkylation_mass=57.0214,
        max_inter_bonds=2,
        ppm_error=15,
    )
    for m in matches:
        print(m)

10973
{'sequence': 'NTDGSTDYGILQINSRWWCNDGR+TPGSRNLCNIPCSALLSSDITASVNCAKK+GTDVQAWIRGCRL', 'ranges': [(7, 9), (9, 12), (15, 18)], 'cysteine_bonds': 2, 'inter_bonds': 2, 'intra_bonds': 0, 'mass': 7159.396725080231, 'error': 0.48568250074936176, 'mods': (Mod(description='cys_alk', mass=57.0214),)}
{'sequence': 'NTDGSTDYGILQINSRWWCNDGRTPGSR+NLCNIPCSALLSSDITASVNCAKK+GTDVQAWIRGCRL', 'ranges': [(7, 10), (10, 12), (15, 18)], 'cysteine_bonds': 2, 'inter_bonds': 2, 'intra_bonds': 0, 'mass': 7159.396725080231, 'error': 0.48568250074936176, 'mods': (Mod(description='cys_alk', mass=57.0214),)}
{'sequence': 'NTDGSTDYGILQINSRWWCNDGRTPGSRNLCNIPCSALLSSDITASVNCAK+K+GTDVQAWIRGCRL', 'ranges': [(7, 11), (11, 12), (15, 18)], 'cysteine_bonds': 2, 'inter_bonds': 2, 'intra_bonds': 0, 'mass': 7159.396725080232, 'error': 0.4856825008763968, 'mods': (Mod(description='cys_alk', mass=57.0214),)}
{'sequence': 'NTDGSTDYGILQINSRWWCNDGRTPGSRNLCNIPCSALLSSDITASVNCAKK+GTDVQAWIR+GCRL', 'ranges': [(7, 12), (15, 16), (16, 18

In [85]:
start_time = time.time()

TAKE = 100

for scan, measurement in list(measurements.items())[:TAKE]:
    for match in sorted(
        precursor_mass_matches(
            peptides,
            measurement,
            alkylation_mass=57.0214,
            max_inter_bonds=2,
            ppm_error=15,
        ),
        key=lambda m: m["sequence"],
    ):
        seq = match["sequence"]
        mods = match["mods"]
        print(f"{scan}: {seq}")

end_time = time.time()
print(f"This takes {end_time - start_time} seconds")

845: CELAAAMK+GCR
846: RHGLDNYR
848: CELAAAMK+GCR
849: CELAAAMK+CK
852: CELAAAMKR+CK
854: CELAAAMKR+GCR
858: NRCK+GCR
889: CELAAAMK+CK
This takes 0.0611879825592041 seconds


In [ ]:

@dataclasses.dataclass
class Run:
    beginning: int
    end: int
    forward: int

    start_modification_mass: float
    break_modification_mass: float
    end_modification_mass: float

    def __init__(
        self,
        beggining: int,
        end: int,
        start_break: bool = False,
        end_break: bool = False,
    ):
        self.beginning = beggining
        self.end = end

        b_ion_mod = -mass.calculate_mass(formula="OH")
        y_ion_mod = 0

        if end > beggining:
            self.forward = 1
            self.start_modification_mass = y_ion_mod if start_break else 0
            self.break_modification_mass = b_ion_mod
            self.end_modification_mass = b_ion_mod if end_break else 0
        else:
            self.forward = -1
            self.start_modification_mass = b_ion_mod if start_break else 0
            self.break_modification_mass = y_ion_mod
            self.end_modification_mass = y_ion_mod if end_break else 0

    def __contains__(self, index: int):
        f = self.forward
        return self.beginning * f <= index * f < self.end * f

    def remove_overlap(self, x: int, y: int):
        x, y = min(x, y), max(x, y)
        if self.forward == 1:
            return Run()

        return Run()


class MultiPeptide:
    _peptides: List[Peptide]
    _cysteine_bonds: List[Tuple[int, int]]
    _ranges: List[Tuple[int, int]]
    _modifications: Dict[str, Tuple[Mod, int]]

    def __getitem__(self, index: int) -> AminoAcid:
        for p in self._peptides:
            aa = p[index]
            if aa is not None:
                return aa

    def bond_partner(self, index: int) -> Union[int, None]:
        for x, y in self._cysteine_bonds:
            if index == x:
                return y
            if index == y:
                return x
        return None

    def __contains__(self, index: int):
        return any(index in range(x, y) for x, y in self._ranges)

    def is_break(self, index: int) -> bool:
        return any(index in range(x + 1, y - 1) for x, y in self._ranges)

    def run_from(self, index: int, forward: bool) -> Run:
        for x, y in self._ranges:
            if x <= index <= y:
                if forward:
                    return Run(index, y)
                else:
                    return Run(index, x - 1)

    @property
    def modded_amino_acids(self) -> Set[str]:
        return {mod.target_aa for mod in self._modifications}

    def count(self, amino_acid: str):
        return sum(p.count(amino_acid) for p in self._peptides)

    def modifications_on(self, amino_acid: str) -> Tuple[Mod, int]:
        return self._modifications[amino_acid]

    def run_mass(self, run: Run) -> float:
        for p in self._peptides:
            if p.beginning == run.beginning and run.end == p.end:
                return p.zwitterion_mass
            elif p.beginning <= run.beginning and run.end <= p.end:
                # MAYBE: Cache this
                return sum(
                    p[i] for i in range(run.beginning, run.end, step=run.forward)
                )


def sort_runs(runs: Tuple[Run, ...]) -> Tuple[Run, ...]:
    return tuple(sorted(runs, key=lambda r: r.beginning))


# TODO: Add min_mass counting
# TODO: Add support for negative-mass modifications
def match_fragments(target, peptide: MultiPeptide, breaks, ppm_error=10):

    result = []
    b_ion_mod = -mass.calculate_mass(formula="OH")
    y_ion_mod = 0

    h2o = mass.calculate_mass(formula="H2O")
    nh3 = mass.calculate_mass(formula="NH3")
    sulphur = mass.calculate_mass(formula="S")
    h2 = mass.calculate_mass(formula="H2")

    def go(
        i,
        current,
        runs: Tuple[Run, ...],
        selection: Tuple[int, ...],
        breaks_left: int,
        bonded_cysteines: Tuple[int, ...],
        broken_cysteines: Tuple[int, ...],
        neutral_losses_count: int,
        modded_amino_acids: Dict[str, int],
    ):
        if len(runs) == 0 and within_bounds(current, target, ppm_error):
            ranges = [
                (x, y) if y > x else (y, x)
                for x, y in zip(selection[::2], (selection + (i,))[1::2])
            ]

            mod_array = []

            final_mass = current

            for aa, count in modded_amino_acids:
                peptide_mod, peptide_mod_count = peptide.modifications_on(aa)

                minimum_mods = max(peptide_mod_count - (peptide.count(aa) - count), 0)
                final_mass += minimum_mods * peptide_mod

                # How many can I have
                maximum_mods = min(peptide_mod_count, count)
                # Optional mods
                for _ in range(maximum_mods - minimum_mods):
                    mod_array.append([(None, 0), peptide_mod])

            for _ in range(neutral_losses_count):
                # MAYBE: Make this more granular? Or ditch this altogether
                mod_array += [
                    ("–H2O neutral loss", -h2o),
                    ("–NH3 neutral loss", -nh3),
                ]

            for c in broken_cysteines:
                symmetric = (
                    j := peptide.bond_partner(c) is not None
                ) and j in broken_cysteines

                # Symmetry breaking
                if symmetric and i > j:
                    continue

                if symmetric:
                    mod_array.append([("-SSH + / or -SH + =S", -h2)])
                else:
                    mod_array.append(
                        [
                            ("-SSH", sulphur),
                            ("- /", -(sulphur + h2)),
                            ("=S", -h2),
                            ("-SH", 0),
                        ]
                    )

            combinations = combine_modifications_2(
                mod_array,
                starting_mass=final_mass,
                target_mass=target,
                ppm_error=ppm_error,
            )

            for modifications in combinations:
                total_mass = final_mass + sum(m.mass for m in modifications)
                result.append(
                    {
                        "ranges": ranges,
                        "mass": total_mass,
                        "error": compute_error(total_mass, target),
                        "mods": modifications,
                    }
                )

        # Nowhere to go next
        if len(runs) == 0 or (len(runs) == 1 and i == runs[0].end):
            return

        # We have to end this run
        # TODO: Prune runs
        if i == runs[0].end and runs[0].forward == 1:
            end_mass_mod = runs[0].end_modification_mass if selection[-1] != i else 0

            pruned_runs = ...

            if i in bonded_cysteines:
                go(
                    runs[1].beginning,
                    current + end_mass_mod,
                    pruned_runs,
                    selection + (i,),
                    breaks_left + 1,  # We're merging
                    bonded_cysteines,
                    broken_cysteines,
                    neutral_losses_count - 1,  # We're merging
                    modded_amino_acids,
                )

            go(
                runs[1].beginning,
                current + end_mass_mod,
                pruned_runs,
                selection + (i,),
                breaks_left,
                bonded_cysteines,
                broken_cysteines,
                neutral_losses_count,
                modded_amino_acids,
            )
        else:
            start_mass_mod = 0
            if i == runs[0].beginning:
                selection += (i,)
                start_mass_mod = runs[0].start_modification_mass

            aa = peptide[i].name
            if aa == "C" and (j := peptide.bond_partner(i) is not None):

                if i < j:
                    # Make sure we don't cross over
                    next_runs = (
                        peptide.run_from(j - 1, forward=False),
                        peptide.run_from(j + 1, forward=True),
                    )

                    # Continue, don't break the SS bond
                    # TODO: Prune runs
                    go(
                        i + runs[0].forward,
                        current + peptide[i].mass + peptide[j].mass + start_mass_mod,
                        sort_runs(runs + next_runs),
                        selection + (j, j + 1),
                        breaks_left,
                        bonded_cysteines + (i, j),
                        broken_cysteines,
                        neutral_losses_count,
                        modded_amino_acids,
                    )

                # Continue, break the SS bond
                go(
                    i + runs[0].forward,
                    current + peptide[i].mass + start_mass_mod,
                    runs,
                    selection,
                    breaks_left - 1,
                    bonded_cysteines,
                    broken_cysteines + (i,),
                    neutral_losses_count,
                    modded_amino_acids,
                )
            else:
                new_modded_amino_acids = modded_amino_acids.copy()
                if aa in peptide.modded_amino_acids:
                    new_modded_amino_acids[aa] += 1

                # Go forward, adding another amino acid
                go(
                    i + runs[0].forward,
                    current + peptide[i].mass + start_mass_mod,
                    runs,
                    selection,
                    breaks_left,
                    bonded_cysteines,
                    broken_cysteines,
                    neutral_losses_count,
                    new_modded_amino_acids,
                )

            # Break the peptide bond, start next run
            go(
                runs[1].beginning,
                current + runs[0].break_modification_mass,
                runs[1:],
                selection + (i,),
                breaks_left - 1,
                bonded_cysteines,
                broken_cysteines,
                neutral_losses_count + 1,
                modded_amino_acids,
            )

    # TODO: Optimize so that we aren't too close to the end
    for beginning in peptide:
        modifier = 0
        if peptide.is_break(beginning):
            breaks -= 1
            modifier = y_ion_mod

        go(
            beginning,
            current=0 + modifier,
            runs=(peptide.run_from(beginning, forward=True),),
            selection=(),
            breaks_left=breaks,
            bonded_cysteines=(),
            broken_cysteines=(),
            neutral_losses_count=0,
            modded_amino_acids={aa: 0 for aa in peptide.modded_amino_acids},
        )

    return result


In [124]:

class MultiP:
    _segments: List[Peptide]
    _disulfide_bond: Dict[int, int] = {}

    def __init__(self, segments: List[Peptide], disulfide_bonds: List[Tuple[int, int]]):
        self._segments = sorted(segments, key=lambda s: s.beginning)
        self.segments = len(segments)
        for c1, c2 in disulfide_bonds:
            self._disulfide_bond[c1] = c2
            self._disulfide_bond[c2] = c1

    def __getitem__(self, index: int) -> AminoAcid:
        for segment in self._segments:
            residue = segment[index]
            if residue is not None:
                return residue

    def __iter__(self):
        for segment in self._segments:
            for i in segment:
                yield i

    def bond_partner(self, residue: int) -> Optional[int]:
        return self._disulfide_bond.get(residue, None)

    def segment(self, residue: int) -> int:
        for i, s in enumerate(self._segments):
            if residue in s:
                return i

    def segment_beginning(self, segment: int) -> int:
        return self._segments[segment].beginning

    def segment_end(self, segment: int) -> int:
        return self._segments[segment].end


In [243]:
# TODO: Target should be with fixed charge (i.e. try all charges in a loop)
# TODO: Peptide should have fixed cysteine bonds (i.e. try all cysteine combinations in a loop)

# TODO: Optimize using binary search
def sort_into(x, xs: Tuple):
    return tuple(sorted(xs + (x,)))


B_ION_MOD = 0
Y_ION_MOD = mass.calculate_mass(formula="H2O")
H2 = mass.calculate_mass(formula="H2")


def fragments(target_mass, peptide: MultiP, allowed_breaks, ppm_error=10):

    result = []

    # TODO: Add mods
    # TODO: Add charges
    def go_run(
        i: int,
        min_end: int,
        max_end: int,
        current_mass: float,
        breaks_left: int,
        new_runs: Tuple[int, ...],
        old_runs: Tuple[int, ...],
        broken_cysteines: Tuple[int, ...],
        pep_bond_breaks_count: int,
        max_i_per_segment: Dict[int, int],
        fragment_start: int,
    ):
        # TODO: Add min/max masses
        if current_mass - err_margin(current_mass, ppm_error) > target_mass:
            # Too heavy, beyond repair, end the whole branch
            return

        if i >= max_end:
            if i > max_end:
                raise AssertionError("This should never happen, i > max_end")

            # We can't grow any longer, end the run
            go(
                i,
                max_i_per_segment,
                current_mass,
                breaks_left,
                broken_cysteines,
                pep_bond_breaks_count,
                new_runs,
                old_runs + (i,),
                fragment_start,
            )
        else:
            residue = peptide[i]

            # This residue is (was) part of a disulfide bond
            if residue.name == "C" and (j := peptide.bond_partner(i) is not None):
                if j in broken_cysteines:
                    # This Cys (i) also has to be added as broken
                    go_run(
                        i + 1,
                        min_end,
                        max_end,
                        current_mass + residue.mass,
                        breaks_left,  # Already added when we were breaking j
                        new_runs,
                        old_runs,
                        broken_cysteines + (i,),
                        pep_bond_breaks_count,
                        max_i_per_segment,
                        fragment_start,
                    )
                else:
                    if breaks_left > 0:
                        # Break the bond
                        go_run(
                            i + 1,
                            min_end,
                            max_end,
                            current_mass + residue.mass,
                            breaks_left - 1,
                            new_runs,
                            old_runs,
                            broken_cysteines + (i,),
                            pep_bond_breaks_count,
                            max_i_per_segment,
                            fragment_start,
                        )

                        if j > fragment_start:
                            # Keep the bond, add new run
                            go_run(
                                i + 1,
                                min_end,
                                max_end,
                                current_mass
                                + residue.mass
                                - H2,  # Subtract H2 for the bond
                                breaks_left,
                                sort_into(j, new_runs),
                                old_runs,
                                broken_cysteines,
                                pep_bond_breaks_count,
                                max_i_per_segment,
                                fragment_start,
                            )

            else:
                # Add current residue, continue the run
                go_run(
                    i + 1,
                    min_end,
                    max_end,
                    current_mass + residue.mass,
                    breaks_left,
                    new_runs,
                    old_runs,
                    broken_cysteines,
                    pep_bond_breaks_count,
                    max_i_per_segment,
                    fragment_start,
                )

            # Break this run and end it
            if i >= min_end and breaks_left > 0:
                # End the run
                go(
                    i,
                    max_i_per_segment,
                    current_mass + B_ION_MOD,
                    breaks_left - 1,
                    broken_cysteines,
                    pep_bond_breaks_count + 1,
                    new_runs,
                    old_runs + (i,),
                    fragment_start,
                )

    def go(
        max_i: int,
        max_i_per_segment: Dict[int, int],
        current: float,
        breaks_left: int,
        broken_cysteines: Tuple[int, ...],
        pep_bond_breaks_count: int,
        runs: Tuple[int, ...],
        old_runs: Tuple[int, ...],
        fragment_start: int,
    ):
        if len(runs) == 0:
            if within_bounds(current, target_mass, ppm_error):
                # TODO: Add more information about current path
                ranges = list(zip(old_runs[::2], old_runs[1::2]))
                result.append(ranges)
            return

        segment = peptide.segment(max_i)
        new_max_i_per_segment = max_i_per_segment.copy()
        new_max_i_per_segment[segment] = max_i

        cys = runs[0]
        current_segment = peptide.segment(cys)
        current_segment_max_i = new_max_i_per_segment[current_segment]

        beg_start = max(
            peptide.segment_beginning(segment), current_segment_max_i, fragment_start
        )
        beg_end = cys

        end_start = cys + 1
        end_end = peptide.segment_end(current_segment)

        at_segment_start = current_segment_max_i == peptide.segment_beginning(segment)
        # TODO: Fix shift optimization
        shift_optim = 0  # not at_segment_start and current_segment_max_i != cys

        for b in range(beg_start + shift_optim, beg_end + 1):
            is_break = b > beg_start

            if not is_break:
                go_run(
                    b,
                    end_start,
                    end_end,
                    current,
                    breaks_left,
                    runs[1:],
                    old_runs + (b,),
                    broken_cysteines,
                    pep_bond_breaks_count,
                    new_max_i_per_segment,
                    fragment_start,
                )

            if is_break and breaks_left > 0:
                go_run(
                    b,
                    end_start,
                    end_end,
                    current + is_break * Y_ION_MOD,
                    breaks_left - is_break,
                    runs[1:],
                    old_runs + (b,),
                    broken_cysteines,
                    pep_bond_breaks_count + is_break,
                    new_max_i_per_segment,
                    fragment_start,
                )

    for b in peptide:
        segment = peptide.segment(b)
        is_break = b != peptide.segment_beginning(segment)

        go(
            0,
            {s: peptide.segment_beginning(s) for s in range(peptide.segments)},
            0,
            allowed_breaks - is_break,
            (),
            is_break,
            (b,),
            (),
            b,
        )

    return result


In [244]:
seq1 = "NTDGSTDYGILQINSRWWCNDGR"
seq2 = "NTDGSTDYGILQINSRWWCNDGR"
multip = MultiP(
    [Peptide(0, len(seq1), seq1, {}), Peptide(30, 30 + len(seq2), seq2, {})],
    [(18, 30 + 18)],
)

target = (
    mass.calculate_mass("NSRWWCND", ion_type="b")
    # + mass.calculate_mass("WWCND", ion_type="b")
    # - H2
)

fragments(
    target,
    multip,
    4,
)

[[(13, 21)], [(43, 51)]]

In [222]:
multip[48]

AminoAcid(name='C', mass=103.00918478471)